In [2]:
import pydicom
import json
import re
import os
import sys
import shutil
import glob
import random
import subprocess
import pathlib
import yaml

In [9]:
# cfg = "C:/Users/smart/Desktop/GitProjects/convsauce/ConvertSource/cfg.test.yml"
cfg = "/Users/brac4g/Desktop/convsauce/ConvertSource/cfg.test.yml"

In [13]:
def read_config(config_file, verbose = False):
    '''
    Reads configuration file and creates a dictionary of search terms for 
    certain modalities provided that BIDS modalities are used as keys. If
    exclusions are provided (via the key 'exclude') then an exclusion list is 
    created. Otherwise, 'exclusion_list' is returned as an empty list. If 
    additional settings are specified, they should be done so via the key
    'settings' to enable writing of additional metadata.
    
    Arguments:
        config_file (string): file path to yaml configuration file.
        verbose (boolean): Prints additional information to screen.
    
    Returns: 
        data_map (dict): Nested dictionary of search terms for BIDS modalities
        exclusion_list (list): List of exclusion terms
        settings_dict (dict): Nested dictionary of metadata terms to write to JSON file(s)
    '''
    
    with open(config_file) as file:
        data_map = yaml.safe_load(file)
        if verbose:
            print("Initialized parameters from configuration file")
        
    if any("exclude" in data_map for element in data_map):
        if verbose:
            print("exclusion option implemented")
        exclusion_list = data_map["exclude"]
        del data_map["exclude"]
    else:
        if verbose:
            print("exclusion option not implemented")
        exclusion_list = list()
        
    if any("settings" in data_map for element in data_map):
        if verbose:
            print("implementing additional settings")
        settings_dict = data_map["settings"]
        del data_map["settings"]
    else:
        if verbose:
            print("no additional settings implemented")
        settings_dict = dict()
        
    return data_map,exclusion_list,settings_dict

In [14]:
search_dict, exclusion_list, param_dict = read_config(cfg,True)

Initialized parameters from configuration file
exclusion option implemented
implementing additional settings


In [15]:
search_dict

{'anat': {'T1w': ['T1', 'T1w', 'TFE'], 'T2w': ['T2', 'T2w', 'TSE']},
 'func': {'bold': {'rest': ['rsfMR', 'rest', 'FFE', 'FEEPI'],
   'visualstrobe': ['vis', 'visual']}},
 'fmap': {'fmap': ['map']},
 'swi': {'swi': ['swi']},
 'dwi': {'dwi': ['diffusion', 'DTI', 'DWI', '6_DIR']}}

In [16]:
exclusion_list

['SURVEY',
 'Reg',
 'SHORT',
 'LONG',
 'MRS',
 'PRESS',
 'DEFAULT',
 'ScreenCapture',
 'PD',
 'ALL',
 'SPECTRO']

In [17]:
param_dict

{'general': {'metadata': {'Manufacturer': 'Philips',
   'ManufacturersModelName': 'Ingenia',
   'InstitutionName': "Cincinnati Children's Hospital Medical Center",
   'ParallelAcquisitionTechnique': 'SENSE'}},
 'func': {'metadata': {'PhaseEncodingDirection': 'j'}}}

In [18]:
# data_dir_par = "C:/Users/smart/Desktop/GitProjects/convsauce/287H_C10/PAR REC"
data_dir_par = "/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC"

In [19]:
# data_dir_dcm = "C:/Users/smart/Desktop/GitProjects/convsauce/IRC287H-8/20171003"
data_dir_dcm = "/Users/brac4g/Desktop/convsauce/IRC287H-8/20171003"

In [21]:
data_dir_nii = "/Users/brac4g/Desktop/convsauce/287H_C10/NIFTI"

In [24]:
def get_dcm_files(dcm_dir):
    '''
    Creates a file list consisting of the first DICOM file in a parent DICOM directory. 
    A file list is then returned.
    
    Arguments:
        dcm_dir (string): Absolute path to parent DICOM data directory

    Returns: 
        dcm_files (list): List of DICOM filenames, complete with their absolute paths.
    '''
    
    # Create directory list
    dcm_dir = os.path.abspath(dcm_dir)
    parent_dcm_dir = os.path.join(dcm_dir,'*')
    dcm_dir_list = glob.glob(parent_dcm_dir, recursive=True)

    # Initilized dcm_file list
    dcm_files = list()
    
    # Iterate through files in the dicom directory list
    for dir_ in dcm_dir_list:
        # print(dir_)
        for root, dirs, files in os.walk(dir_):
            # print(files[0])
            tmp_dcm_file = files[0] # only need the first dicom file
            tmp_dcm_dir = root
            tmp_file = os.path.join(tmp_dcm_dir, tmp_dcm_file)

            dcm_files.append(tmp_file)
            break

    return dcm_files

In [27]:
def create_file_list(data_dir, file_ext="", order="size"):
    '''
    Creates a file list by globbing a directory for a specific file
    extension and sorting by some determined order. A file list is 
    then returned
    
    Arguments:
        data_dir (string): Absolute path to data directory (must be a directory dump of image data)
        file_ext (string): File extension to glob. Built-in options include:
            - 'par' or 'PAR': Searches for PAR headers
            - 'dcm' or 'DICOM': Searches for DICOM directories, then searches for one file from each DICOM directory
            - 'nii', or 'Nifti': Searches for nifti files (including gzipped nifti files)
        order (string): Order to sort the list. Valid options are: 'size' and 'time':
            - 'size': sorts by file size in ascending order (default)
            - 'time': sorts by file modification time in ascending order
            - 'none': no sorting is applied and the list is generated as the system finds the files
    
    Returns: 
        file_list (list): List of filenames, complete with their absolute paths.
    '''
    
    # Check file extension
    if file_ext != "":
        if file_ext.upper() == "PAR" or file_ext.upper() == "REC":
            file_ext = "PAR"
            file_ext = f".{file_ext.upper()}"
        elif file_ext.lower() == "dcm" or file_ext.upper() == "DICOM":
            file_ext = "dcm"
            file_ext = f".{file_ext.lower()}"
        elif file_ext.lower() == "nii" or file_ext.lower() == "nifti":
            file_ext = "nii"
            file_ext = f".{file_ext.lower()}*" # Add wildcard for globbling gzipped files
        else:
            file_ext = f".{file_ext}"
    
    # Check sort order
    if order.lower() == "size":
        order_key = os.path.getsize
    elif order.lower() == "time":
        order_key = os.path.getmtime
    elif order.lower() == "none":
        order_key=None
    else:
        order_key = os.path.getsize
        print("Unrecognized keyword option. Using default.")
    
    # Create file list
    if file_ext == ".dcm":
        file_list = sorted(get_dcm_files(data_dir), key=order_key, reverse=False)
    elif file_ext != ".dcm":
        file_names = os.path.join(data_dir, f"*{file_ext}")
        file_list = sorted(glob.glob(file_names, recursive=True), key=order_key, reverse=False)
    
    return file_list

In [28]:
par_file_list = create_file_list(data_dir=data_dir_par,file_ext="par")
dcm_file_list = create_file_list(data_dir=data_dir_dcm,file_ext="dcm")
nii_file_list = create_file_list(data_dir=data_dir_nii,file_ext="nii")

In [30]:
par_file_list

['/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_WIP_NEONATAL_SURVEY_2_1.PAR',
 '/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_WIP_NEONATAL_SURVEY_1_1.PAR',
 '/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_WIP_NEONATAL_SURVEY_14_1.PAR',
 '/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_WIP_NEONATAL_SURVEY_16_1.PAR',
 '/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_WIP_NEONATAL_SURVEY_13_1.PAR',
 '/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_WIP_NEONATAL_SURVEY_17_1.PAR',
 '/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_isoReg_-_WIP_DTI_6DIR_B800_12_6.PAR',
 '/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_faReg_-_WIP_DTI_6DIR_B800_12_4.PAR',
 '/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_SAG_4_5.PAR',
 '/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_SAG_15_5.PAR',
 '/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_AXIAL__4_4.PAR',
 '/Users/brac4g/Desktop/convsauce/2

In [31]:
dcm_file_list

['/Users/brac4g/Desktop/convsauce/IRC287H-8/20171003/1101_rsfMRI_MB6_SENSE_1_fat_shift_P_017100310465322437/MR1101027463.dcm',
 '/Users/brac4g/Desktop/convsauce/IRC287H-8/20171003/0_DEFAULT_PS_SERIES_2017100310463791022/PR0000000001.dcm',
 '/Users/brac4g/Desktop/convsauce/IRC287H-8/20171003/1701_WM_SV_PRESS_35_017100311174543840/MR1701000001.dcm',
 '/Users/brac4g/Desktop/convsauce/IRC287H-8/20171003/201_SAG_T1W_3D_Y_INNER_TI_1100_017100310184810020/MR0201000137.dcm',
 '/Users/brac4g/Desktop/convsauce/IRC287H-8/20171003/303_CORONAL_2017100310262626000/MR0303000091.dcm',
 '/Users/brac4g/Desktop/convsauce/IRC287H-8/20171003/0_DEFAULT_PS_SERIES_2017100310374358016/PR0000000001.dcm']

In [32]:
nii_file_list

['/Users/brac4g/Desktop/convsauce/287H_C10/NIFTI/287H_C10_WIP_Philips_GRE_Map_SENSE_10_1.nii.gz',
 '/Users/brac4g/Desktop/convsauce/287H_C10/NIFTI/287H_C10_WIP_T2W_TSE_AXIAL_NEONATE_NSA1_15_1.nii.gz',
 '/Users/brac4g/Desktop/convsauce/287H_C10/NIFTI/287H_C10_WIP_T2W_TSE_AXIAL_NEONATE_NSA1_4_1.nii.gz',
 '/Users/brac4g/Desktop/convsauce/287H_C10/NIFTI/287H_C10_WIP_SAG_T1W_3D_Y_INNER_TI_1100_3_1.nii.gz',
 '/Users/brac4g/Desktop/convsauce/287H_C10/NIFTI/287H_C10_CORONAL__15_3.nii.gz',
 '/Users/brac4g/Desktop/convsauce/287H_C10/NIFTI/287H_C10_AXIAL__15_4.nii.gz',
 '/Users/brac4g/Desktop/convsauce/287H_C10/NIFTI/287H_C10_SAG_15_5.nii.gz',
 '/Users/brac4g/Desktop/convsauce/287H_C10/NIFTI/287H_C10_AXIAL__3_3.nii.gz',
 '/Users/brac4g/Desktop/convsauce/287H_C10/NIFTI/287H_C10_SAG_4_5.nii.gz',
 '/Users/brac4g/Desktop/convsauce/287H_C10/NIFTI/287H_C10_AXIAL__4_4.nii.gz',
 '/Users/brac4g/Desktop/convsauce/287H_C10/NIFTI/287H_C10_CORONAL__4_3.nii.gz',
 '/Users/brac4g/Desktop/convsauce/287H_C10/NIFTI

In [84]:
def file_exclude(file_list, data_dir, exclusion_list = [], verbose = False):
    '''
    Excludes files from the conversion process by removing filenames
    that contain words that match those found in the 'exclusion_list'
    from the 'read_config' function - should any files need/want to be 
    excluded.
    
    If 'exclusion_list' is empty, then the original 'file_list' is returned.
    
    Arguments:
        file_list (list): List of filenames
        data_dir (string): Absolute path to parent directory that contains the image data
        exclusion_list (list): List of words to be matched. Filenames that contain these words will be excluded.
        verbose (bool): Boolean - True or False.
    
    Returns: 
        currated_list (list): Currated list of filenames, with unwanted filenames removed.
    '''
            
    # Check file extension in file list
    if 'dcm' in file_list[0]:
        file_ext = "dcm"
        file_ext = f".{file_ext.lower()}"
    elif 'PAR' in file_list[0]:
        file_ext = "PAR"
        file_ext = f".{file_ext.upper()}"
    elif 'nii' in file_list[0]:
        file_ext = "nii"
        file_ext = f".{file_ext.lower()}*" # Add wildcard for globbling gzipped files
    else:
        file_ext = ""
        file_ext = f".{file_ext.lower()}"
    
    # create set of lists
    file_set = set(file_list)
    
    # create empty sets
    currated_set = set()
    exclusion_set = set()
    
    if len(exclusion_list) == 0:
        currated_set = file_set
    else:
        for file in exclusion_list:
            if file_ext == '.dcm':
                dir_ = os.path.join(data_dir, f"*{file}*",f"*{file_ext}")
            else:
                dir_ = os.path.join(data_dir, f"*{file}*{file_ext}")
            f_names = glob.glob(dir_, recursive=True)        
            f_names_set = set(f_names)
            exclusion_set.update(f_names_set)
            
        currated_set = file_set.difference(exclusion_set)

    currated_list = list(currated_set)
    
    return currated_list

In [85]:
par_file_list_currated = file_exclude(par_file_list,data_dir_par,exclusion_list)
dcm_file_list_currated = file_exclude(dcm_file_list,data_dir_dcm,exclusion_list)
nii_file_list_currated = file_exclude(nii_file_list,data_dir_nii,exclusion_list)

In [86]:
par_file_list_currated

['/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_WIP_SAG_T1W_3D_Y_INNER_TI_1100_3_1.PAR',
 '/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_WIP_DTI_B2000_A68_FAT_SHIFT_P_MB2_SENSE_2_TE88_21_1.PAR',
 '/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_WIP_rsfMRI_NR1_MB3_SENSE_1_fat_shift_A_7_1.PAR',
 '/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_WIP_rsfMRI_400M_MB3_SENSE_1_fat_shift_P_9_1.PAR',
 '/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_WIP_Philips_GRE_Map_SENSE_10_1.PAR',
 '/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_WIP_36DIR_DTI_B800__P_SENSE_NO__MB_MAX_GRAD_6_1.PAR',
 '/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_WIP_DTI_6DIR_B800_12_1.PAR',
 '/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_AXIAL__4_4.PAR',
 '/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_WIP_6_DIR_B0_A_TE88_6DYN_20_1.PAR',
 '/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_WIP_T2W_TSE_AXIAL_NEONATE_NSA

In [87]:
len(par_file_list_currated)

21

In [88]:
dcm_file_list_currated

['/Users/brac4g/Desktop/convsauce/IRC287H-8/20171003/303_CORONAL_2017100310262626000/MR0303000091.dcm',
 '/Users/brac4g/Desktop/convsauce/IRC287H-8/20171003/201_SAG_T1W_3D_Y_INNER_TI_1100_017100310184810020/MR0201000137.dcm',
 '/Users/brac4g/Desktop/convsauce/IRC287H-8/20171003/1101_rsfMRI_MB6_SENSE_1_fat_shift_P_017100310465322437/MR1101027463.dcm']

In [89]:
len(dcm_file_list_currated)

3

In [90]:
nii_file_list_currated

['/Users/brac4g/Desktop/convsauce/287H_C10/NIFTI/287H_C10_AXIAL__15_4.nii.gz',
 '/Users/brac4g/Desktop/convsauce/287H_C10/NIFTI/287H_C10_SAG_15_5.nii.gz',
 '/Users/brac4g/Desktop/convsauce/287H_C10/NIFTI/287H_C10_AXIAL__4_4.nii.gz',
 '/Users/brac4g/Desktop/convsauce/287H_C10/NIFTI/287H_C10_SAG_4_5.nii.gz',
 '/Users/brac4g/Desktop/convsauce/287H_C10/NIFTI/287H_C10_WIP_T2W_TSE_AXIAL_NEONATE_NSA1_4_1.nii.gz',
 '/Users/brac4g/Desktop/convsauce/287H_C10/NIFTI/287H_C10_CORONAL__3_2.nii.gz',
 '/Users/brac4g/Desktop/convsauce/287H_C10/NIFTI/287H_C10_CORONAL__4_3.nii.gz',
 '/Users/brac4g/Desktop/convsauce/287H_C10/NIFTI/287H_C10_CORONAL__15_3.nii.gz',
 '/Users/brac4g/Desktop/convsauce/287H_C10/NIFTI/287H_C10_WIP_SAG_T1W_3D_Y_INNER_TI_1100_3_1.nii.gz',
 '/Users/brac4g/Desktop/convsauce/287H_C10/NIFTI/287H_C10_AXIAL__3_3.nii.gz',
 '/Users/brac4g/Desktop/convsauce/287H_C10/NIFTI/287H_C10_WIP_Philips_GRE_Map_SENSE_10_1.nii.gz',
 '/Users/brac4g/Desktop/convsauce/287H_C10/NIFTI/287H_C10_WIP_T2W_TSE_A

In [91]:
len(nii_file_list_currated)

12

In [92]:
search_dict

{'anat': {'T1w': ['T1', 'T1w', 'TFE'], 'T2w': ['T2', 'T2w', 'TSE']},
 'func': {'bold': {'rest': ['rsfMR', 'rest', 'FFE', 'FEEPI'],
   'visualstrobe': ['vis', 'visual']}},
 'fmap': {'fmap': ['map']},
 'swi': {'swi': ['swi']},
 'dwi': {'dwi': ['diffusion', 'DTI', 'DWI', '6_DIR']}}

In [93]:
def str_in_substr(sub_str_,str_):
    '''
    DEPRECATED: Should only be used if config_file uses comma separated
        lists to denote search terms.
    
    Searches a (longer) string using a comma separated string 
    consisting of substrings. Returns 'True' or 'False' if any part
    of the substring is found within the larger string.
    
    Example:
        str_in_substr('T1,TFE','sub_T1_image_file') would return True.
        str_in_substr('T2,TSE','sub_T1_image_file') would return False.
    
    Arguments:
        sub_str_ (string): Substring used for matching.
        str_ (string): Larger string to be searched for matches within substring.
    
    Returns: 
        bool_var (bool): Boolean - True or False
    '''
    
    bool_var = False
    
    for word in sub_str_.split(","):
        if any(word in str_ for element in str_):
            bool_var = True
            
    return bool_var

In [121]:
def list_in_substr(list_,str_):
    '''
    Searches a string using a list that contains substrings. 
    Returns 'True' or 'False' if any elements of the list are 
    found within the string.
    
    Example:
        list_in_substr('['T1','TFE']','sub_T1_image_file') would return True.
        list_in_substr('['T2','TSE']','sub_T1_image_file') would return False.
    
    Arguments:
        list_ (string): list containing strings used for matching.
        str_ (string): Larger string to be searched for matches within substring.
    
    Returns: 
        bool_var (bool): Boolean - True or False
    '''
    
    bool_var = False
    
    for word in list_:
        if any(word.lower() in str_.lower() for element in str_.lower()):
            bool_var = True
            
    return bool_var

In [101]:
def convert_modality(dictionary, file):
    '''
    Converts an image file and extracts information from the filename
    (such as the modality). 
    
    Note: This function is still undergoing active development.
    
    Arguments:
        dictionary (dict): Nested dictionary from the 'read_config' function
        file (string): Filename with absolute filepath
    
    Returns: 
        None
    '''
    
    mod_found = False
    
    for key,item in dictionary.items():
        for dict_key,dict_item in dictionary[key].items():
            if isinstance(dict_item,list):
                if list_in_substr(dict_item,file):
                    mod_found = True
                    print(f"{key} - {dict_key}: {dict_item}")
                    if mod_found:
                        break
            elif isinstance(dict_item,dict):
                tmp_dict = dictionary[key]
                for d_key,d_item in tmp_dict[dict_key].items():
                    if list_in_substr(d_item,file):
                        mod_found = True
                        print(f"{key} - {dict_key} - {d_key}: {d_item}")
                        if mod_found:
                            break
                        
    if not mod_found:
        print("unknown modality")
    
    return None

In [102]:
for idx,file in enumerate(par_file_list_currated):
    # print("test_parse(search_list,file)")
    convert_modality(search_dict,file)
    print(idx+1)
    
# print(idx+1)

anat - T1w: ['T1', 'T1w', 'TFE']
1
dwi - dwi: ['diffusion', 'DTI', 'DWI', '6_DIR']
2
func - bold - rest: ['rsfMR', 'rest', 'FFE', 'FEEPI']
3
func - bold - rest: ['rsfMR', 'rest', 'FFE', 'FEEPI']
4
unknown modality
5
dwi - dwi: ['diffusion', 'DTI', 'DWI', '6_DIR']
6
dwi - dwi: ['diffusion', 'DTI', 'DWI', '6_DIR']
7
unknown modality
8
dwi - dwi: ['diffusion', 'DTI', 'DWI', '6_DIR']
9
anat - T2w: ['T2', 'T2w', 'TSE']
10
unknown modality
11
unknown modality
12
unknown modality
13
unknown modality
14
func - bold - rest: ['rsfMR', 'rest', 'FFE', 'FEEPI']
15
unknown modality
16
unknown modality
17
dwi - dwi: ['diffusion', 'DTI', 'DWI', '6_DIR']
18
unknown modality
19
unknown modality
20
anat - T2w: ['T2', 'T2w', 'TSE']
21


In [103]:
for idx,file in enumerate(dcm_file_list_currated):
    # print("test_parse(search_list,file)")
    convert_modality(search_dict,file)
    print(idx+1)
    
# print(idx+1)

unknown modality
1
anat - T1w: ['T1', 'T1w', 'TFE']
2
func - bold - rest: ['rsfMR', 'rest', 'FFE', 'FEEPI']
3


In [104]:
for idx,file in enumerate(nii_file_list_currated):
    # print("test_parse(search_list,file)")
    convert_modality(search_dict,file)
    print(idx+1)
    
# print(idx+1)

unknown modality
1
unknown modality
2
unknown modality
3
unknown modality
4
anat - T2w: ['T2', 'T2w', 'TSE']
5
unknown modality
6
unknown modality
7
unknown modality
8
anat - T1w: ['T1', 'T1w', 'TFE']
9
unknown modality
10
unknown modality
11
anat - T2w: ['T2', 'T2w', 'TSE']
12


In [136]:
def get_dcm_scan_tech(dictionary, dcm_file):
    '''
    working doc-string
    '''
    
    mod_found = False
    
    # Load DICOM data and read header
    ds = pydicom.dcmread(dcm_file)
    
    # Parse DICOM header for Scan Technique used
    dcm_scan_tech_str = str(ds[0x2001,0x1020])
    
    for key,item in dictionary.items():
        for dict_key,dict_item in dictionary[key].items():
            if isinstance(dict_item,list):
                if list_in_substr(dict_item,dcm_scan_tech_str):
                    mod_found = True
                    print(f"{key} - {dict_key}: {dict_item}")
                    if mod_found:
                        break
            elif isinstance(dict_item,dict):
                tmp_dict = dictionary[key]
                for d_key,d_item in tmp_dict[dict_key].items():
                    if list_in_substr(d_item,dcm_scan_tech_str):
                        mod_found = True
                        print(f"{key} - {dict_key} - {d_key}: {d_item}")
                        if mod_found:
                            break
                            
        if mod_found:
            break
    
    # Secondary look in the case Private Field (2001, 1020) [Scanning Technique Description MR] is empty
    if not mod_found:
        dcm_fields = ['SeriesDescription', 'ImageType', 'ProtocolName'] # Define list of DICOM header fields
        
        for dcm_field in dcm_fields:
            dcm_scan_tech_str = str(eval(f"ds.{dcm_field}"))
            
            for key,item in dictionary.items():
                for dict_key,dict_item in dictionary[key].items():
                    if isinstance(dict_item,list):
                        if list_in_substr(dict_item,dcm_scan_tech_str):
                            mod_found = True
                            print(f"{key} - {dict_key}: {dict_item}")
                            if mod_found:
                                break
                    elif isinstance(dict_item,dict):
                        tmp_dict = dictionary[key]
                        for d_key,d_item in tmp_dict[dict_key].items():
                            if list_in_substr(d_item,dcm_scan_tech_str):
                                mod_found = True
                                print(f"{key} - {dict_key} - {d_key}: {d_item}")
                                if mod_found:
                                    break

            if mod_found:
                break
                
    if not mod_found:
        print("unknown modality")

In [135]:
get_dcm_scan_tech(search_dict,dcm_file_list_currated[2])

func - bold - rest: ['rsfMR', 'rest', 'FFE', 'FEEPI']


In [107]:
ds = pydicom.dcmread(dcm_file_list_currated[0])

In [108]:
ds

(0008, 0000) Group Length                        UL: 614
(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'PROJECTION IMAGE', 'M', 'SE']
(0008, 0012) Instance Creation Date              DA: '20171003'
(0008, 0013) Instance Creation Time              TM: '112749.800'
(0008, 0014) Instance Creator UID                UI: 1.3.46.670589.11.89.5
(0008, 0016) SOP Class UID                       UI: MR Image Storage
(0008, 0018) SOP Instance UID                    UI: 1.3.46.670589.11.71329.5.0.10456.2017100310262751090
(0008, 0020) Study Date                          DA: '20171003'
(0008, 0021) Series Date                         DA: '20171003'
(0008, 0022) Acquisition Date                    DA: '20171003'
(0008, 0023) Content Date                        DA: '20171003'
(0008, 0030) Study Time                          TM: '095546'
(0008, 0031) Series Time                         TM: '102626.265'
(000

In [119]:
'TSE'.lower() in str(ds[0x2001,0x1020]).lower()

True

In [123]:
ds.SeriesDescription

'CORONAL'

In [126]:
ds.ImageType

['ORIGINAL', 'PRIMARY', 'PROJECTION IMAGE', 'M', 'SE']

In [127]:
ds.ProtocolName

'CORONAL'

In [128]:
t = "ProtocolName"

In [129]:
eval(f"ds.{t}")

'CORONAL'

In [224]:
len(file_list_currated)

21

In [111]:
verbose = False

In [112]:
if verbose:
    print("True")

In [30]:
# way to implement function [file list function]
# 1. make file list
# 2. read config file
# 3. exclude files from file list

In [31]:
# way to implelement conversion function [file conversion]
# 1. read in file list
# 2. read in config file
# 3. convert files in list

In [23]:
ex1 = (exL[1])
ex_main = ex1[0]
ex_tmp = ex1[1].split(",")
ex_tmp

['T2', 'T2w', 'TSE']

In [11]:
ex_tmp

['T2', 'T2w', 'TSE']

In [12]:
# use this to check if filename is in ex_tmp
if 'T1' in ex_tmp or 'TFE' in ex_tmp:
    print("True")

In [13]:
f1 = "C:/Users/smart/Desktop/GitProjects/convsauce/287H_C10/PAR_REC/287H_C10_WIP_T2W_TSE_AXIAL_NEONATE_NSA1_4_1.PAR"

In [14]:
if f1 in ex_tmp:
    print("True")

In [15]:
any(ele in f1 for ele in ex_tmp)

True

In [16]:
if any(ele in f1 for ele in ex_tmp):
    print("True")

True


In [17]:
exL[2]

['bold', {'rest': 'rsfMR,rest', 'visualstrobe': 'vis,visual'}]

In [18]:
type(exL[2])

list

In [19]:
ex1 = (exL[2])
ex_main = ex1[0]
ex1[1]
# ex_tmp = ex1[1].split(",")
# ex_tmp

{'rest': 'rsfMR,rest', 'visualstrobe': 'vis,visual'}

In [20]:
ex1[1]

{'rest': 'rsfMR,rest', 'visualstrobe': 'vis,visual'}

In [21]:
for key, values in ex1[1].items():
    for value in values.split(','):
        print(f"{key}: {value}")

rest: rsfMR
rest: rest
visualstrobe: vis
visualstrobe: visual


#### `TaskName` JSON file appending funtion

In [4]:
# task_name = ""
task_name = "visualstrobe"

In [5]:
if task_name == "":
    print("task_name is empty")
else:
    print(f"task_name is: {task_name}")

task_name is: visualstrobe


In [1]:
nii_file = "c:/Users/smart/Desktop/GitProjects/convsauce/BIDS/rawdata/sub-C10/ses-001/func/sub-C10_ses-001_task-rest_acq-PA_run-01_bold.nii.gz"

In [2]:
import os
import nibabel as nib
import numpy as np

In [3]:
os.path.exists(nii_file)

True

In [4]:
img = nib.load(nii_file)

In [5]:
img

In [6]:
img.header.get_data_shape()

(64, 64, 45, 400)

In [7]:
type(img.header.get_data_shape())

tuple

In [9]:
frames = img.header.get_data_shape()

In [10]:
frames

(64, 64, 45, 400)

In [13]:
type(frames[3])

int